In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [12]:
import pandas as pd
autentica = pd.read_excel('ls.xlsx')
login = str(autentica['login'][0])
senha = str(autentica['senha'][0])

display(autentica)

pedidos = pd.read_excel('consolidacao/pedidos.xlsx')
pedidos = pedidos[pedidos['acao'].str.contains('pedido no suap')]
pedidos.index = [x for x in range(1, len(pedidos)+1)]
pedidos['i'] = [x for x in range(1, len(pedidos)+1)]
# pedidos = pedidos.iloc[0:6]

qtd = pedidos[['i', 'diferenca']]
dict_qtd = qtd.to_dict('records')
ped = pedidos[['i', 'codigo']]
dict_ped = ped.to_dict('records')

indisponivel = "indisponivel"

,login,senha
0,2184579,sis1986%
1,48836,48836


In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
import time

def abrir_suap():
    
    global nav
    nav = webdriver.Chrome()
    nav.get("https://suap.ifpe.edu.br/accounts/login/?next=/")
    # inserindo usuário
    nav.find_element_by_xpath('//*[@id="id_username"]').click()
    nav.find_element_by_xpath('//*[@id="id_username"]').send_keys(login)

    # inserindo senha 
    nav.find_element_by_xpath('//*[@id="id_password"]').click()
    nav.find_element_by_xpath('//*[@id="id_password"]').send_keys(senha)

    nav.find_element_by_xpath('//*[@id="ok"]').click()

In [7]:
df_elemento_despesa = pd.DataFrame()
df_temp = pd.DataFrame(columns=('value', 'ed'))
df_temp.loc[0] = ['14', 'esportes']
# display(df_temp)

opcoes = []
valores = []

def varrer_ed():
#     abrir_suap()
    nav.get("https://suap.ifpe.edu.br/almoxarifado/form_busca/materialconsumo/")
        
    valor_ed = nav.find_element_by_id('id_elemento_despesa')
    options = [x for x in valor_ed.find_elements_by_tag_name('option')]
    
    for i in options:
        # print(i.get_attribute('value')+" "+i.text)
        opcoes.append(i.text)
        valores.append(i.get_attribute('value'))
#         print(f'{i.text}-{i.get_attribute("value")}')

    df_elemento_despesa['value'] = valores
    df_elemento_despesa['ed']= opcoes
    
#     display(df_elemento_despesa)

In [8]:
def get_qtd():
    
    abrir_suap()
    
    df_tabela = pd.DataFrame()
    tabela = []
    codigos = []
    descricoes = []
    quantidades = []
    valores = []
    elemento = []
    
    try:
        pag = 1
        saida = True
        while saida:
                
            nav.get(f'https://suap.ifpe.edu.br/almoxarifado/form_busca/materialconsumo/?page={pag}&material=&elemento_despesa=&estoque_tipo=emcampus&unidade_medida=')
            celula = nav.find_element_by_tag_name('td')
                
            if celula:
                lista = nav.find_element_by_tag_name('table')
                td = lista.find_elements_by_tag_name('td')
                
                for i in td:
                    tabela.append(i.text)
                
                pag += 1
            else:
                saida = False
                
    except NoSuchElementException:
        print('acabaram as páginas')
        
    for cod in range(0, len(tabela), 7):
        codigos.append(tabela[cod])

    for des in range(1, len(tabela), 7):
        descricoes.append(tabela[des])

    for qtd in range(4, len(tabela), 7):
        quantidades.append(tabela[qtd])
    
    for val in range(5, len(tabela), 7):
        valores.append(tabela[val])
        
    for ed in range(3, len(tabela), 7):
        elemento.append(tabela[ed])

    df_tabela['codigo'] = codigos
    df_tabela['descricao'] = descricoes
    
    df_tabela['qtd'] = quantidades
    df_tabela['qtd'] = pd.to_numeric(df_tabela['qtd'])
    
    df_tabela['vlr_total'] = valores
    df_tabela['vlr_total'] = df_tabela['vlr_total'].str[2:]
    df_tabela['vlr_total'] = pd.to_numeric(df_tabela['vlr_total'])
    
    df_tabela['vlr_unt'] = df_tabela['vlr_total'] / df_tabela['qtd']
    df_tabela['vlr_unt'] = df_tabela['vlr_unt'].round(2)
    
    df_tabela['ed'] = elemento
    
    df_tabela.to_excel('relatorio/relatorio.xlsx', index=False)
    display(df_tabela)
    
    nav.quit()


In [10]:
def set_pedidos():
    abrir_suap()

    nav.get('https://suap.ifpe.edu.br/almoxarifado/form_requisicao_pessoa_pedido/')

    # preenchendo campo do requisitante
    nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/fieldset/div[2]/input[1]').send_keys('Andrea Dacal')
    time.sleep(3)
    nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/fieldset/div[2]/input[1]').send_keys(Keys.DOWN)
    nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/fieldset/div[2]/input[1]').send_keys(Keys.ENTER)

    # preenchento campo do pedido do material

    for i in dict_ped:
        nav.find_element_by_xpath(f'/html/body/div[1]/div[3]/form/div[1]/fieldset/div[{i["i"]}]/input[1]').send_keys(i['codigo'])
        time.sleep(2)
        nav.find_element_by_xpath(f'/html/body/div[1]/div[3]/form/div[1]/fieldset/div[{i["i"]}]/input[1]').send_keys(Keys.DOWN)
        nav.find_element_by_xpath(f'/html/body/div[1]/div[3]/form/div[1]/fieldset/div[{i["i"]}]/input[1]').send_keys(Keys.ENTER)
        time.sleep(1)
        nav.find_element_by_name('addItem').click()

    for i in dict_qtd:    
        nav.find_element_by_xpath(f'/html/body/div[1]/div[3]/form/div[1]/fieldset/div[{i["i"]}]/input[3]').send_keys(i['diferenca'])

acabaram as páginas


,codigo,descricao,qtd,vlr_total,vlr_unt,ed
0,30.04.3,NITROGÊNIO LÍQUIDO - GÁS LIQUEFEITO REFRIGERAD...,1,418.64,418.64,04 - GAS E OUTROS MATERIAIS ENGARRAFADOS
1,30.07.18,EXTRATO DE TOMATE.,4,22.00,5.50,07 - GENEROS DE ALIMENTACAO
2,30.07.193,"ALMONDEGA, CARNE BOVINA.",754,10000.60,13.26,07 - GENEROS DE ALIMENTACAO
3,30.07.39,"SAL REFINADO IODADO, PCT 1KG.",33,37.95,1.15,07 - GENEROS DE ALIMENTACAO
4,30.07.404,"Massa para lasanha, ideal para ir direto ao fo...",12,42.00,3.50,07 - GENEROS DE ALIMENTACAO
...,...,...,...,...,...,...
1010,30.63.27,"SACOLA DE EVENTO TIPO ECOBAG, dimensões 35 x 4...",264,1380.72,5.23,63 - SERVIÇOS GRAFICOS E EDITORIAIS
1011,30.63.41,"CANECA DE PLÁSTICO - CAPACIDADE 400ML, PERSONA...",181,369.24,2.04,63 - SERVIÇOS GRAFICOS E EDITORIAIS
1012,30.63.44,Estojo em nailon cores variadas e impressão se...,45,197.55,4.39,63 - SERVIÇOS GRAFICOS E EDITORIAIS
1013,30.63.52,AGENDA IFPE. SERVIDOR/ANUAL,652,8723.76,13.38,63 - SERVIÇOS GRAFICOS E EDITORIAIS
